# Proyecto Naive Bayes Algorithm

In [225]:
# Librería para la declaración y uso de Data Frames:
import pandas as pd

# Librería para poder realizar la partición del conjunto de datos:
from sklearn.model_selection import train_test_split

# Librería para poder utilizar un vectorizador de textos:
from sklearn.feature_extraction.text import CountVectorizer

# Librería para poder utilizar los algoritmos de Navie Bayes:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

# Librería para poder realizar el análisis de rendimiento:
from sklearn.metrics import classification_report

# Librería para poder utilizar la técnica de optimización de parámetros:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Librería para poder utilizar el modelo de RandomForest:
from sklearn.ensemble import RandomForestClassifier

# Libería para poder generar números aleatorios:
from scipy.stats import randint

## Paso 1 - Lectura de Datos:

En primer lugar, es necesario **leer y guardar la información** en una variable para poder empezar a trabajar con ella.

Para ello, se ha guardado el archivo con todos los datos en la ruta: */workspaces/naive-bayes-clara-ab/data/raw/playstore_reviews.csv* y se ha cargado en un Data Frame:

In [209]:
# Lectura del CSV con los datos, dada la ruta donde se guarda el archivo:
df = pd.read_csv ('/workspaces/naive-bayes-clara-ab/data/raw/playstore_reviews.csv');

# Configuración de pandas para mostrar todas las columnas del DataFrame sin truncarlas al visualizarlo
pd.set_option('display.max_columns', None);

# Se muestran las 5 primeras filas del Data Frame
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


Una vez se ha cargado correctamente la información en el Data Frame, es interesante evaluar **qué tipo y cuánta información se tiene**. Para ello, se recurre al atributo `.shape` del Data Frame:

In [210]:
# Se utiliza el atributo shape del Data Frame para conocer cuánta información está cargada:
print (f" El conjunto de datos cuenta con {df.shape[0]} comentarios con total de {df.shape[1]} tipos de información sobre ellos");

 El conjunto de datos cuenta con 891 comentarios con total de 3 tipos de información sobre ellos


## Paso 2 - Análisis Exploratorio de Datos:

Como se anticipaba en la lectura de las intrucciones del proyecto, el **conjunto de datos parte de tres variables**: 

- `package_name` : Nombre de la aplicación móvil (variable categórica)

- `review` : Comentario sobre la aplicación móvil (variable categórica)

- `polarity` : Variable objetivo que categoriza con un 0 un comentario positivo y con 1 un comentario positivo

Tal y como se indica en dichas instrucciones, de las **variables predictoras** (`package_name` y  `review`) solo **interesa** realmente el **comentario**, dado que para clasificar si un comentario es o no positivo dependerá de su contenido y no de en qué aplicación se ha escrito. 

Esto implica que hay que **eliminar** la columna `package_name`, para lo que se va a utilizar el método `.drop()`

In [211]:
# Se elimina la variable que no aporta información sobre el objetivo:
df = df.drop(['package_name'], axis = 1);

# Se comprueba que se ha eliminado correctamente:
print (f" El conjunto de datos cuenta con {df.shape[0]} comentarios con total de {df.shape[1]} tipos de información sobre ellos");

 El conjunto de datos cuenta con 891 comentarios con total de 2 tipos de información sobre ellos


Finalmente, también será necesario asegurar que los **datos se encuentren en el formato correcto**,  es decir, **no se puede trabajar con texto plano**, si no que es necesario procesarlo **eliminando espacios** ( `.strip()` ) y **conviertiendo** todas las letras a **minúscula** `.lower()` . 

Este paso solo es necesario realizarlo en la columna de `review` dado que es la **única que contiene texto**:

In [212]:
df ['review'] = df ['review'].str.strip().str.lower()

En esta ocasión **no es necesario realizar un Análisis Exploratorio de Datos** (EDA) en tanto que la **variable predictora** con la que se cuenta es de **tipo texto**. En caso de tener un número más elevado de variables independientes y que fuesen de tipo numérico, sí sería necesario realizar este paso. 

## Paso 3 - Partición del Conjunto de Datos:

Como se acaba de mencionar, la **variable independiente es el contenido del comentario** `review` mientras que la **variable a predecir es si es o no positivo** dicho comentario, `polarity`. 

Por esta razón, en primer lugar, se va a realizar esta **separación entre variable independiente y dependiente**:

In [213]:
# Se separa la variable dependiente: 
y = df ['polarity']

# Se separa la variable independiente (en este caso es un simple array al solo ser una variable):
x = df ['review']
print(x)

0      privacy at least put some option appear offlin...
1      messenger issues ever since the last update, i...
2      profile any time my wife or anybody has more t...
3      the new features suck for those of us who don'...
4      forced reload on uploading pic on replying com...
                             ...                        
886    loved it i loooooooooooooovvved it because it ...
887    all time legendary game the birthday party lev...
888    ads are way to heavy listen to the bad reviews...
889    fun works perfectly well. ads aren't as annoyi...
890    they're everywhere i see angry birds everywher...
Name: review, Length: 891, dtype: object


Ahora que ya se han separado las variables se puede proceder a realizar la **partición del conjunto de datos**, teniendo una parte para **entrenar al modelo** (*train*) y otra para **probarlo** (*test*). De esta forma, se puede evaluar el rendimiento sin inferir en su **capacidad predictiva**.

Un detalle a destacar es que, como en este caso solo se cuenta con **una variable independiente**, no se trabaja con una matriz. Esto impica que la nomenclatura utilizada ya **no es una X_train (maýuscula)** como se venía usando en todos los proyectos hasta ahora, **si no x_train (minúscula)**. 

In [214]:
# Se realiza la partición, explicitando el tamaño del test set:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42);

## Paso 4 - Preparación de la información:

Los **algoritmos de Machine Learning no pueden trabajar directamente con datos en formato de texto**. Por esta razón, en general, se suele realizar una **codificación de variables categóricas a numéricas** como parte del EDA. Sin embargo, como se ha explicado antes, en este caso la única variable predictora con la que se cuenta es en formato texto por lo que un EDA carecía de sentido. 

El **objetivo principal de predicción** en este caso es saber si un c**omentario es o no positivo**. Es por ello por lo que se está trabajando con el **contenido de esta opinión**, lo que implica que el **interés recae en analizar las palabras** como tal. 

En este sentido, se va a utilizar una **instancia de la clase** `CountVectorizer` de forma que se pueda **convertir el texto en una matrix** donde **cada línea haga referencia a un comentario** y **cada columna a las palabras únicas** encontradas en el conjunto de entrenamiento. Así se consigue **información de la frecuencia de cada palabra** dentro de cada comentario, es decir, cada celda de la matriz muestra cuántas veces aparece esa palabra en el comentario correspondiente. 


En resumen, el primer paso es **declarar la instancia del vectorizador** para **entrenarlo con el *train* set** y, posteriormente, **aplicarlo sobre el *test* set**. 

In [215]:
# Se declara una instancia del vectorizador: 
vectorizer = CountVectorizer();

# Se entrena y aplica la vectorización sobre el train set:
x_train_vec = vectorizer.fit_transform(x_train);

# Se aplica el vectorizador entrenador sobre el test set:
x_test_vec = vectorizer.transform (x_test);

## Paso 5 - Estudio del Algoritmo Naive Bayes:

Ahora que ya se tiene el conjunto de datos preparado, se puede pasar a **obtener las predicciones** sobre si un comentario es o no postivo. 

Para ello, se van a utilizar el algoritmo **Naive Bayes**. Este tipo de **algoritmo de clasificación** está basado en el Teorema de Bayes y asume que **todas las características** que participan en la predicción son **independientes entre sí**. Su **mayor eficiencia** se da en problemas de **clasificación de texto**, como es el caso actual.

En este estudio se van a explorar **tres variantes del algoritmo**: **BernouilliNB**, **MultinomialNB** y **GaussianNB**. Cada implementación va a ser evaluada para comparar su rendimiento.

Sin embargo, antes de nada, se va a comprobar **cómo de balanceado se encuentra el conjunto de datos**, es decir, que el número de comentarios positivos y negativos es similar.

In [216]:
df['polarity'].value_counts()

polarity
0    584
1    307
Name: count, dtype: int64

Como se puede comprobar, el conjunto de datos **no está balanceado**, al tener muchos más datos de comentarios negativos que positivos. 

Esto es algo que **afecta a las métricas que analizan la capacidad predictiva del modelo**, en tanto que es mucho más probable que acierten en un sentido que en el otro y, por lo tanto, se deberá **tener en cuenta para el análisis de resultados**. 


### Paso 5.1 - Modelo de tipo Bernoulli:

La variante del algoritmo de Naive Bayes, el **modelo Bernoulli**, está diseñada para trabajar con **problemas de clasificación don de las características son binarias**. En situaciones de **procesamiento de texto**, como es el que ocupa, se podría utilizar para **modelar la ausencia o presencia de palabras claves** en el documento. 

En este caso, no se sabe exactamente **qué criterio se ha seguido para catalogar** un comentario como positivo o no. Sin embargo, si se asume que hay una serie de **palabras clave** que hacen que el comentario se pueda clasificar de un lado o del otro, este **modelo podría ser muy útil y obtener un buen rendimiento**. 

Para poder utilizarlo, se va a **instanciar y entrenar el clasificador** de la clase `BernouilliNB` y, para **evaluar su rendimiento**, se utilizará la función `classification_repot()`:

In [218]:
# Se inicializa y entrena el clasificador Naive Bayes Bernouilli:
bernoulli_model = BernoulliNB().fit(x_train_vec, y_train);

# Se realizan predicciones en el conjunto de prueba:
y_pred_bernouilli = bernoulli_model.predict(x_test_vec);

# Se evalúa el rendimiento del modelo:
print(classification_report(y_test, y_pred_bernouilli))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       126
           1       0.81      0.64      0.72        53

    accuracy                           0.85       179
   macro avg       0.84      0.79      0.81       179
weighted avg       0.85      0.85      0.84       179



El modelo muestra un buen desempeño general, con una precisión general del 85%, aunque su rendimiento puede estar influido por el desbalance de datos comentado anteriormente. 

Si se evalúa la **Sensibilidad**, *Recall*, hay una **gran diferencia entre cada uno de los dos tipo de comentarios**. Por un lado, de las veces que realmente **hubo un comentario NO positivo** (negativo), el modelo lo identificó como tal en el **94% de los casos**, muy por encima del **64% para el caso de los comentarios positivos**. Si se estudia el *F1-Score* pasa algo similar, siendo **mejor el rendimiento para la clase 0** que para la 1.

En general, es evidente que el **modelo se ve sesgado hacia la clase mayoritaria**, aunque su rendimiento general es **aceptable pero mejorable**.

### Paso 5.2 - Modelo de tipo Multinomial:

El **modelo Multinomial** es otra variante del algoritmo Naive Bayes que se utiliza principalmente cuando las **características son representadas como frecuencias de ocurrencia**, es decir, exactamente lo que se tiene en este caso. 

A diferencia de Bernouilli, que se enfoca en la presencia o ausencia de ciertas palabras clave, Multinomial asume que las **caracterísiticas siguen una distribución multinomial**, lo ideal cuando los datos consisten en recuentos de ocurrencias. Por lo que **este modelo parece el ideal** para el caso que ocupa.

Para poder utilizarlo, se va a **instanciar y entrenar el clasificador** de la clase `MultinomialNB` y, para **evaluar su rendimiento**, se utilizará la función `classification_repot()`:



In [220]:
# Se inicializa y entrena el clasificador Naive Bayes Multinomial:
multinomial_model = MultinomialNB().fit(x_train_vec, y_train)

# Se realizan predicciones en el conjunto de prueba:
y_pred_multinomial = multinomial_model.predict(x_test_vec)

# Se evalúa el rendimiento del modelo:
print(classification_report(y_test, y_pred_multinomial))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90       126
           1       0.84      0.58      0.69        53

    accuracy                           0.84       179
   macro avg       0.84      0.77      0.79       179
weighted avg       0.84      0.84      0.83       179



El modelo MultinomialNB presenta una **precisión global del 84%** pero, de nuevo, refleja un **sesgo hacia la clase mayoritaria**. 

Para el caso de los **comentarios NO positivos** (negativos) el modelo tiene una **precisión del 85%** y una **Sensibilidad del 95%**, lo que indica una **identificación muy acertada** de este tipo de comentarios. Sin embargo, para la clase 1, **comentarios positivos**, de cada 100 comentarios positivos, solo se ha **identificado correctamente 58 de ellos**, lo que denota una **leve bajada** con respecto al modelo de Bernouilli.

En general, ambos modelos presentan un **comportamiento similar**, enfrentándose a **problemas derivados del desbalance de clases**. 

### Paso 5.3 - Modelo de tipo Gaussiano:

Por último, el modelo Gausiano es otra vairante del algoritmo Naive Bayes, principalmente utilizado en casos donde las características de los datos siguen una distribución normal. Esto implica que cada característica se describe mediante una media y una desviación estándar que el modelo puede utilizar para calcular las probabilidades de cada clase. 

Por este motivo, el modelo de tipo gaussiano no parece que vaya a ser la mejor opción en tanto que las características de los datos son no son continuas. 

De todas formas, se va a **instanciar y entrenar el clasificador** de la clase `GaussianNB` y, para **evaluar su rendimiento**, se utilizará la función `classification_repot()`:

In [221]:
# Se inicializa y entrena el clasificador Naive Bayes Gaussian:
gaussian_model = GaussianNB().fit(x_train_vec.toarray(), y_train);

# Se realizan predicciones en el conjunto de prueba;
y_pred_gaussian = gaussian_model.predict(x_test_vec.toarray())

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred_gaussian))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86       126
           1       0.70      0.60      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179



Este modelo presenta una **precisión global del 80%**, lo cual ya es **inferior a los resultados obtenidos con Bernouilli y Multinomial**. 

Al igual que en los casos anteriores, los **comentarios negativos tienen buenas métricas** aunque son **las peores obtenidas hasta el momento**. Por otro lado, si se atiende a la clase minoritaria, **los comentarios positivos**, el **rendimiento es inferior** y en la mayoría de los casos está **por debajo que los conseguidos** mediente los otros dos modelos. 

Teniendo en centa todo este análisis, parece que los **dos modelos que funcionan mejor** son tanto **Bernouilli** como **Multinomial**, tal y como **se esperaba desde un primer momento**.

Para poder sacar el **máximo partido de ellos** se ha utilizado `GridSearch` como **técnica de optimización de hiperparámetros**. En este caso no se va a hacer una explicación detallada al ya haberlo hecho en detalle en proyectos anteriores:

In [222]:
# Se inicializa  el clasificador Naive Bayes Bernouilli:
bernoulli_model = BernoulliNB();

# Se define el espacio de búsqueda los hiperparámetros para explorar
param_grid = {
    'alpha': [0.5, 1.0, 2.0, 5.0, 10.0],  # Controla cuánto se suavizan las probabilidades de características no observadas
    'binarize': [0.0, 0.1, 0.5, 1.0]  # Define el umbral para convertir características en valores binarios
};

# Se configura el GridSearchCV para encontrar el mejor conjunto de hiperparámetros:
grid_search = GridSearchCV(bernoulli_model, param_grid, cv=5, scoring='accuracy');

# Se entrena el modelo con los mejores hiperparámetros:
grid_search.fit(x_train_vec, y_train);

# Se imprimen los mejores parámetros:
print(f"Mejores parámetros encontrados: {grid_search.best_params_}");

# Se usa el mejor modelo encontrado por GridSearchCV
best_bernoulli_model = grid_search.best_estimator_;

# Se evalúa el rendimiento en el conjunto de prueba:
y_pred_bernoulli = best_bernoulli_model.predict(x_test_vec);
print(classification_report(y_test, y_pred_bernoulli));


Mejores parámetros encontrados: {'alpha': 1.0, 'binarize': 0.0}
              precision    recall  f1-score   support

           0       0.86      0.94      0.90       126
           1       0.81      0.64      0.72        53

    accuracy                           0.85       179
   macro avg       0.84      0.79      0.81       179
weighted avg       0.85      0.85      0.84       179



In [223]:
# Se inicializa  el clasificador Naive Bayes Multinomial:
multinomial_model = MultinomialNB();

# Se define el espacio de búsqueda los hiperparámetros para explorar:
param_grid = {
    'alpha': [0.5, 1.0, 2.0, 5.0, 10.0],  # Controla cuánto se suavizan las probabilidades de características no observadas
    'fit_prior': [True, False]  #  Indica si se ajustan las probabilidades a priori de las clases según su frecuencia en los datos
}

# Se configura el GridSearchCV para encontrar el mejor conjunto de hiperparámetros
grid_search = GridSearchCV(multinomial_model, param_grid, cv=5, scoring='accuracy');

# Se entrena el modelo con los mejores hiperparámetros:
grid_search.fit(x_train_vec, y_train);

# Se imprimen los mejores parámetros encontrados:
print(f"Mejores parámetros encontrados: {grid_search.best_params_}");

# Se usa el mejor modelo encontrado por GridSearchCV:
best_multinomial_model = grid_search.best_estimator_;

# Se evalúa el rendimiento en el conjunto de prueba;
y_pred_multinomial = best_multinomial_model.predict(x_test_vec);
print(classification_report(y_test, y_pred_multinomial));


Mejores parámetros encontrados: {'alpha': 1.0, 'fit_prior': False}
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       126
           1       0.80      0.60      0.69        53

    accuracy                           0.84       179
   macro avg       0.82      0.77      0.79       179
weighted avg       0.83      0.84      0.83       179



Los **resultados obtenidos son muy similares a los que se han conseguido con el modelo sin optimizar**, lo que puede estar muy relacionado con la simpleza del conjunto de datos al tener solo una variable independiente. 

Teniendo en cuenta todo este análisis, parece que el **modelo de tipo Bernoulli es el que mejor rendimiento presenta**. En un principio **se había pensado que el modelo de tipo Multinomial sería el más efectivo**, debido a la naturaleza de la variable independiente, sin embargo, **BernouilliNB ha presentado un buen equilibrio en las métricas**, sobretodo en la clase 0 y ha obtenido los valores más altos para la clase minoritaria, clase 1. 

# Paso 6 - Estudio con Random Forest:

En el intento de obtener una **mejor predicción** para el conjunto de datos bajo estudio, se va a utilizar otro **algoritmo de clasificación**, `RandomForest` y se va a evaluar su rendimiento. 

Para este caso, se van a emplear también **técnicas de optimización de hiperparámetros**, siendo en este caso la técnica escogida `RandomizedSearch` al ser la que **mejor funcionó en proyectos anteriores** (aunque no tiene por qué volver a ocurrir en este caso).

Se van a probar **3 espacios de búsqueda distintos** y se analizará el que mejor rendimiento consiga aportar:

In [226]:
# Se define el modelo de forma general: 
random_forest = RandomForestClassifier(random_state = 42);

# Se define el espacio de búsqueda de hiperparámetros: 
parameters = {
    'n_estimators': randint(30, 100),
    'max_depth': [10, 12, 14, 16],
    'min_samples_leaf': randint(2, 8),
    'max_features': [int(x_train_vec.shape[1] * p) for p in [0.5, 0.7, 0.8, 1.0]]
};

# Se configura la búsqueda en malla con la validación cruzada:
random_search = RandomizedSearchCV(estimator = random_forest, param_distributions = parameters, n_iter = 100, cv = 5, n_jobs = -1, verbose = 1, random_state = 42);

# Se ajusta el modelo con los datos de entrenamiento
random_search.fit(x_train_vec, y_train);

# Se muestran los mejores hiperparámetros encontrados los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {random_search.best_params_}");

# Se evalúa el mejor modelo:
best_random_forest = random_search.best_estimator_;

# Se obtiene la predicción para el conjunto de train:
y_pred_train_forest = best_random_forest.predict(x_train_vec);

# Se obtiene la predicción para el conjunto de test:
y_pred_test_forest = best_random_forest.predict(x_test_vec);

# Se obtienen las métricas:
print(classification_report(y_test, y_pred_test_forest));

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Mejores hiperparámetros: {'max_depth': 12, 'max_features': 1776, 'min_samples_leaf': 3, 'n_estimators': 73}
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       126
           1       0.65      0.58      0.61        53

    accuracy                           0.78       179
   macro avg       0.74      0.72      0.73       179
weighted avg       0.78      0.78      0.78       179



In [228]:
# Se define el modelo de forma general: 
random_forest = RandomForestClassifier(random_state = 42);

# Se define el espacio de búsqueda de hiperparámetros: 
parameters = {
    'n_estimators': randint(50, 100),
    'max_depth': [10, 20, 40, 60],
    'min_samples_leaf': randint(2, 8),
    'max_features': [int(x_train_vec.shape[1] * p) for p in [0.5, 0.7, 0.8, 1.0]]
};

# Se configura la búsqueda en malla con la validación cruzada:
random_search = RandomizedSearchCV(estimator = random_forest, param_distributions = parameters, n_iter = 100, cv = 5, n_jobs = -1, verbose = 1, random_state = 42);

# Se ajusta el modelo con los datos de entrenamiento
random_search.fit(x_train_vec, y_train);

# Se muestran los mejores hiperparámetros encontrados los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {random_search.best_params_}");

# Se evalúa el mejor modelo:
best_random_forest = random_search.best_estimator_;

# Se obtiene la predicción para el conjunto de train:
y_pred_train_forest = best_random_forest.predict(x_train_vec);

# Se obtiene la predicción para el conjunto de test:
y_pred_test_forest = best_random_forest.predict(x_test_vec);

# Se obtienen las métricas:
print(classification_report(y_test, y_pred_test_forest));

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Mejores hiperparámetros: {'max_depth': 60, 'max_features': 1776, 'min_samples_leaf': 2, 'n_estimators': 55}
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       126
           1       0.61      0.66      0.64        53

    accuracy                           0.78       179
   macro avg       0.73      0.74      0.74       179
weighted avg       0.78      0.78      0.78       179



In [229]:
# Se define el modelo de forma general: 
random_forest = RandomForestClassifier(random_state = 42);

# Se define el espacio de búsqueda de hiperparámetros: 
parameters = {
    'n_estimators': randint(10, 30),
    'max_depth': [10, 15, 20, 25],
    'min_samples_leaf': randint(2, 8),
    'max_features': [int(x_train_vec.shape[1] * p) for p in [0.5, 0.7, 0.8, 1.0]]
};

# Se configura la búsqueda en malla con la validación cruzada:
random_search = RandomizedSearchCV(estimator = random_forest, param_distributions = parameters, n_iter = 100, cv = 5, n_jobs = -1, verbose = 1, random_state = 42);

# Se ajusta el modelo con los datos de entrenamiento
random_search.fit(x_train_vec, y_train);

# Se muestran los mejores hiperparámetros encontrados los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {random_search.best_params_}");

# Se evalúa el mejor modelo:
best_random_forest = random_search.best_estimator_;

# Se obtiene la predicción para el conjunto de train:
y_pred_train_forest = best_random_forest.predict(x_train_vec);

# Se obtiene la predicción para el conjunto de test:
y_pred_test_forest = best_random_forest.predict(x_test_vec);

# Se obtienen las métricas:
print(classification_report(y_test, y_pred_test_forest));

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Mejores hiperparámetros: {'max_depth': 25, 'max_features': 1776, 'min_samples_leaf': 4, 'n_estimators': 23}
              precision    recall  f1-score   support

           0       0.84      0.85      0.84       126
           1       0.63      0.60      0.62        53

    accuracy                           0.78       179
   macro avg       0.73      0.73      0.73       179
weighted avg       0.77      0.78      0.78       179



Los **resultados obtenidos** para los tres casos probados son **bastante parecidos** pero **peores que los conseguidos con el modelo de tipo Bernouilli**. 

La **caída más notable** ha sido en la **precisión de la clase minoritaria**, habiendo bajado en más de un **15%**, seguida de la precisión de la mayoritaria, con una caída del 9%. 

El **resto de métricas han sido levemente peores** con el algoritmo RandomForest, obteniendo solo una **precisión del 78% frente al 84% de Bernouilli**. 